In [1]:
import asimov_database as ad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pr = ad.ParquetReader()



class Causality:
    def __init__(self, dol, wdo, date):
        self.date = date
        self.dol = dol
        self.wdo = wdo
    
    def _files(self):

        #Carrega e Trata os arquivos
        level_data = pr.get_parquet(self.dol, self.date, "level-book")
        self.events = pr.get_parquet(self.dol, self.date, "events")["incremental"].read().to_pandas()
        snap = pr.get_parquet(self.dol, self.date, "events")["snapshot"].read().to_pandas()

        level_data_wdo = pr.get_parquet(self.wdo, self.date, "level-book")
        self.events_wdo = pr.get_parquet(self.wdo, self.date, "events")["incremental"].read().to_pandas()
        snap_wdo = pr.get_parquet(self.wdo, self.date, "events")["snapshot"].read().to_pandas()

        bid_price = level_data["bid_price"].read().to_pandas()
        bid_quantity = level_data["bid_quantity"].read().to_pandas()
        bid_inc_code = level_data["bid_inc_code"].read().to_pandas()

        ask_price = level_data["ask_price"].read().to_pandas()
        ask_quantity = level_data["ask_quantity"].read().to_pandas()
        ask_inc_code = level_data["ask_inc_code"].read().to_pandas()

        bid_price_wdo= level_data_wdo["bid_price"].read().to_pandas()
        bid_quantity_wdo = level_data_wdo["bid_quantity"].read().to_pandas()
        bid_inc_code_wdo = level_data_wdo["bid_inc_code"].read().to_pandas()

        ask_price_wdo = level_data_wdo["ask_price"].read().to_pandas()
        ask_quantity_wdo = level_data_wdo["ask_quantity"].read().to_pandas()
        ask_inc_code_wdo = level_data_wdo["ask_inc_code"].read().to_pandas()

        bid_wdo = bid_price_wdo[["bid_0"]].copy()
        bid_quantity_wdo.columns = ["bid_quantity_wdo_{}".format(i) for i in range(bid_quantity_wdo.shape[1])]
        bid_wdo["bid_quantity_wdo_0"] = bid_quantity_wdo["bid_quantity_wdo_0"]
        bid_wdo["i_bid_wdo"] = bid_inc_code_wdo[0]#.apply(lambda x: "B_" + str(x))
        bid_wdo.rename({'bid_0' : 'bid_wdo_0'}, axis = 1, inplace = True )

        bid = bid_price[["bid_0"]].copy()
        bid_quantity.columns = ["bid_quantity_{}".format(i) for i in range(bid_quantity.shape[1])]
        bid["bid_quantity_0"] = bid_quantity["bid_quantity_0"]
        bid["i_bid"] = bid_inc_code[0]

        ask = ask_price[["ask_0"]].copy()
        ask_quantity.columns = ["ask_quantity_{}".format(i) for i in range(ask_quantity.shape[1])]
        ask["ask_quantity_0"] = ask_quantity["ask_quantity_0"]
        ask["i_ask"] =ask_inc_code[0]#.apply(lambda x: "A_" + str(x))

        ask_wdo = ask_price_wdo[["ask_0"]].copy()

        ask_wdo.rename({'ask_0':'ask_wdo_0'}, axis = 1, inplace= True)

        ask_quantity_wdo.columns = ["ask_quantity_wdo_{}".format(i) for i in range(ask_quantity_wdo.shape[1])]
        ask_wdo["ask_quantity_wdo_0"] = ask_quantity_wdo["ask_quantity_wdo_0"]
        ask_wdo["i_ask_wdo"] =ask_inc_code_wdo[0]

        self.events.reset_index(inplace=True)
        self.events_wdo.reset_index(inplace=True)

        #Procura por atualizações provenientes da bolsa e retira da versão final
        valor_dol = np.setdiff1d(self.events['i'].values, bid_inc_code[0].values)
        valor_dol   

        retira_dol = np.setdiff1d(valor_dol,ask_inc_code[0].values)
        retira_dol

        valor_wdo_ask = np.setdiff1d(self.events_wdo['i'].values, bid_inc_code_wdo[0].values)
        valor_wdo_ask

        retira_wdo_ask = np.setdiff1d(valor_wdo_ask,ask_inc_code_wdo[0].values)
        retira_wdo_ask

        valor_wdo_bid = np.setdiff1d(self.events_wdo['i'].values, ask_inc_code_wdo[0].values)
        valor_wdo_bid

        retira_wdo_bid = np.setdiff1d(valor_wdo_bid, bid_inc_code_wdo[0].values)
        retira_wdo_bid

        if len(retira_dol) != 0:
            for i in retira_dol:
                self.events = self.events[self.events['i'] != i]


        if len(retira_wdo_bid) != 0:
            for i in retira_wdo_bid:
                self.events_wdo = self.events_wdo[self.events_wdo['i'] != i]
        
        #Cria os arquivos de bid e ask
        bid["msg_seq_num"] = self.events[self.events["side"] != "A"]["msg_seq_num"].values
        ask["msg_seq_num"] = self.events[self.events["side"] != "B"]["msg_seq_num"].values
        bid_wdo["msg_seq_num"] = self.events_wdo[self.events_wdo["side"] != "A"]["msg_seq_num"].values
        ask_wdo["msg_seq_num"] = self.events_wdo[self.events_wdo["side"] != "B"]["msg_seq_num"].values

        #Cria a versao final ja preenchendo os espacos vazios com os correspondentes valores
        df = pd.concat([ask, bid]).sort_values(by='msg_seq_num', kind = 'mergesort').reset_index()
        dl = pd.concat([ask_wdo, bid_wdo]).sort_values(by='msg_seq_num', kind = 'mergesort').reset_index()

        df['symbol'] = 'DOL'
        dl['symbol'] = 'WDO'
        df.ffill(inplace=True)
        df.dropna(inplace=True)
        dl.ffill(inplace=True)
        dl.dropna(inplace=True)
        
        total = pd.concat([df, dl]).sort_values(by='msg_seq_num', kind = 'mergesort').reset_index()

        total.ffill(inplace=True)
        total.dropna(inplace=True)

        total_ = total[['index','msg_seq_num', 'ask_0', 'ask_quantity_0', 'bid_0', 'bid_quantity_0', 'ask_wdo_0', 'ask_quantity_wdo_0', 'bid_wdo_0', 'bid_quantity_wdo_0']][ (total['ask_0'] != 0) & (total['bid_0'] != 0)]

        total_['spread_dol'] = total['ask_0'] - total['bid_0']
        total_['spread_wdo'] = total['ask_wdo_0'] - total['bid_wdo_0']
        
        #Procura pelo primeiro momento de mercado estável
        begin = np.where( (total_['spread_wdo'] == 0.5) & (total_['spread_dol'] == 0.5) & (total_['ask_0'] == total_['ask_wdo_0']) & (total_['bid_0'] == total_['bid_wdo_0']))[0][0]

        total_.rename({'index' : 'timestamp'}, axis = 1, inplace = True)
        
        # Inicia análise no primeiro momento de estabilidade
        slice_ = total_.loc[begin:]

        #Calcula mid price
        slice_['mid_price'] = (slice_['ask_0'] + slice_['bid_0'])/2.0
        slice_['mid_price_wdo'] = (slice_['ask_wdo_0'] + slice_['bid_wdo_0'])/2.0

        #Filtras msg_seq_number e observa apenas o ultimo evento ocorrido
        slice_['repeat'] = False
        slice_['repeat'][slice_['msg_seq_num'] != slice_['msg_seq_num'].shift(-1)] = True
        slice_  = slice_[slice_['repeat']]
        slice_ = slice_.drop(['repeat'], axis =1).reset_index()
        
        #Procura primeiro instante de spread aberto
        index_ = np.where( (slice_['spread_dol'] != 0.5) | (slice_['spread_wdo'] != 0.5) )[0]

        #Procura as posicoes no vetor anterior, onde os momento de spread aberto nao sao consecutivos
        position = np.where(np.diff(np.where( (slice_['spread_dol'] != 0.5) | (slice_['spread_wdo'] != 0.5) )) != 1)[1] +1

        #Procura os indices que no dataframe correspondem ao primeiro momento dos eventos de instabilidade
        A = []
        for i in position:
            A.append(index_[i])

        ############################################################
        #Cria um dict com o evento de distorçao e o evento anterior
        frame = pd.DataFrame()
        dict_ = {}
        j=0
        for i in A:
            dict_[j] = frame.append(slice_.loc[i-1:i])
            j+=1

        dict_aux ={}
        for i in dict_.keys():
            dict_aux[i] = dict_[i].diff()

        for i in dict_.keys():
            dict_aux[i]['spread_dol'] = dict_[i]['spread_dol']
            dict_aux[i]['spread_wdo'] = dict_[i]['spread_wdo']  
            dict_aux[i]['msg_seq_num'] = dict_[i]['msg_seq_num']
            
        #########################################################

        # distortion = pd.concat(dict_).reset_index()

        # mover = distortion.dropna()[['ask_0', 'bid_0', 'ask_wdo_0', 'bid_wdo_0' ]]
        # mover[mover['ask_0'] != 0].count()
        # mover[mover['bid_0'] != 0].count()
        # mover[mover['ask_wdo_0'] != 0].count()
        # mover[mover['bid_wdo_0'] != 0].count()
        
        ############################################################
        #Cria um dict com o evento de distorçao e o evento anterior
        #Cria um dict com o evento de distorçao e 300 eventos posteriores e o evento anterior
#         frame_next = pd.DataFrame()
#         dict_next = {}
#         j=0
#         for i in A:
#             dict_next[j] = frame_next.append(slice_.loc[i-1:i+300])
#             j+=1

#         dict_next_aux = {}
#         for i in dict_next.keys():
#             dict_next_aux[i] = dict_next[i].diff()
#         #########################################################

#         for i in dict_next.keys():
#             dict_next_aux[i]['spread_dol'] = dict_next[i]['spread_dol']
#             dict_next_aux[i]['spread_wdo'] = dict_next[i]['spread_wdo']   

#         return dict_next_aux
        return dict_aux

        # distortion_next = pd.concat(dict_next_aux).reset_index()

    # Cria novas colunas do dataframe com valores que correspondem ao trend do movimento
    def func(self, dataframe):
        dataframe['movement_dol'] = [-1 if x < 0 else 0 if x == 0 else 1 for x in dataframe['mid_price']]
        dataframe['movement_wdo'] = [-1 if x < 0 else 0 if x == 0 else 1 for x in dataframe['mid_price_wdo']]
#         # dataframe = self._files()
#         dataframe.loc[dataframe['mid_price'] < 0, 'movement_dol'] = -1
#         dataframe.loc[dataframe['mid_price']  == 0, 'movement_dol'] = 0
#         dataframe.loc[dataframe['mid_price']  > 0, 'movement_dol'] = 1
#         dataframe.loc[dataframe['mid_price_wdo'] < 0, 'movement_wdo'] = -1
#         dataframe.loc[dataframe['mid_price_wdo']  == 0, 'movement_wdo'] = 0
#         dataframe.loc[dataframe['mid_price_wdo']  > 0, 'movement_wdo'] = 1
        return dataframe 

    def organiza(self):

        dict_next_aux = self._files()
        
        # Aplica a funcao anterior ao dataframe
        df = {}
        for i in dict_next_aux.keys():
            df[i] = self.func(dict_next_aux[i])[['movement_dol', 'movement_wdo',  'mid_price', 'mid_price_wdo', 'msg_seq_num']].dropna()
        
#         for i in dict_next_aux.keys():
#             df[i].dropna(inplace = True)

        # Procura no dict de dataframes o primeiro momento onde todos os comportamentos ocorrem no evento de distorçao 
        for i in df.keys():
            if (len(np.where(df[i]['movement_wdo'] == 1)[0]) != 0)  and (len(np.where(df[i]['movement_wdo'] == -1)[0]) != 0) and (len(np.where(df[i]['movement_dol'] == 1)[0]) != 0) and (len(np.where(df[i]['movement_dol'] == -1)[0]) != 0):
                break
        
        a = i
        return df, a

    def operacoes(self): 
        
        df, begin_ = self.organiza()
        self.aux_dol = 0
        self.aux_wdo = 0
        self.follow_up  = 0
        self.not_follow_up = 0
        self.follow_down = 0
        self.not_follow_down = 0
        self.follow_up_wdo  = 0
        self.not_follow_up_wdo = 0
        self.follow_down_wdo = 0
        self.not_follow_down_wdo = 0
#         self.cum_sum_wdo_WDO = {}
#         self.cum_sum_dol_WDO = {}
#         self.cum_sum_wdo_DOL = {}
#         self.cum_sum_dol_DOL = {}

        ###Inicia do primeiro momento nao vazio, ate o final do dict
        for i in range(begin_,max(df.keys())):
#             self.cum_sum_wdo_WDO[i] = {}
#             self.cum_sum_dol_WDO[i] = {}
#             self.cum_sum_wdo_DOL[i] = {}
#             self.cum_sum_dol_DOL[i] = {}
            df[i].reset_index(inplace=True) #Reinicia index para fazer slice
            for j in range(max(len(np.where(df[i]['movement_wdo'] == 1)[0]), len(np.where(df[i]['movement_wdo'] == -1)[0]), len(np.where(df[i]['movement_dol'] == 1)[0]), len(np.where(df[i]['movement_dol'] == -1)[0]))):
                a_ = np.where(df[i]['movement_wdo'] == 1)[0]
                b_ = np.where(df[i]['movement_wdo'] == -1)[0]
                c_ = np.where(df[i]['movement_dol'] == 1)[0]
                d_ = np.where(df[i]['movement_dol'] == -1)[0]
                if (len(a_) != 0)  and (len(b_) != 0) and (len(c_) != 0) and (len(d_) != 0):
                    ## Checa para ver se o WDO foi responsavel pelo primeiro movimento 
                    if min(a_[0], b_[0]) <  min(c_[0], d_[0]):
                        self.aux_wdo+=1
                        ## Checa para ver se o primeiro movimento foi up ou down 
                        if a_[0] < b_[0]:
                            steps = b_[0] # Determina o tamanho do slice no DataFrame
                            if c_[0] < steps:
                                self.follow_up_wdo +=1
#                                 self.cum_sum_wdo_WDO[i][j] = df[i]['mid_price_wdo'].loc[a_[0]:steps-1].sum()
#                                 self.cum_sum_dol_WDO[i][j] = df[i]['mid_price'].loc[c_[0]:steps-1].sum()
                            else:
                                self.not_follow_up_wdo +=1  
                        else:
                            steps = a_[0]
                            if d_[0] < steps:
                                self.follow_down_wdo +=1
#                                 self.cum_sum_wdo_WDO[i][j] = df[i]['mid_price_wdo'].loc[b_[0]:steps-1].sum()
#                                 self.cum_sum_dol_WDO[i][j] = df[i]['mid_price'].loc[d_[0]:steps-1].sum()
                            else:
                                self.not_follow_down_wdo +=1
                    else:
                        self.aux_dol+=1
                        if c_[0] < d_[0]:
                            steps = d_[0]
                            if a_[0] < steps:
                                self.follow_up +=1
#                                 self.cum_sum_wdo_DOL[i][j] = df[i]['mid_price_wdo'].loc[a_[0]:steps-1].sum()
#                                 self.cum_sum_dol_DOL[i][j] = df[i]['mid_price'].loc[c_[0]:steps-1].sum()
                            else:
                                self.not_follow_up +=1
                        else:
                            steps = c_[0]
                            if b_[0] < steps:
                                self.follow_down +=1
#                                 self.cum_sum_wdo_DOL[i][j] = df[i]['mid_price_wdo'].loc[b_[0]:steps-1].sum()
#                                 self.cum_sum_dol_DOL[i][j] = df[i]['mid_price'].loc[d_[0]:steps-1].sum()
                            else:
                                self.not_follow_down +=1

                df[i] = df[i].loc[steps:]
#                 df[i].dropna(inplace = True)





In [2]:
def plot():
    N = 2
    DOL_Following = (e.follow_up_wdo/(e.follow_up_wdo+e.not_follow_up_wdo),  e.follow_up/(e.follow_up+e.not_follow_up))
    WDO_Following = (e.follow_down_wdo/(e.follow_down_wdo+e.not_follow_down_wdo), e.follow_down/(e.follow_down+e.not_follow_down))

    DOL_FollowingStd = np.std([e.follow_up_wdo/(e.follow_up_wdo+e.not_follow_up_wdo), e.follow_down_wdo/(e.follow_down_wdo+e.not_follow_down_wdo)])

    WDO_FollowingStd = np.std([ e.follow_up/(e.follow_up+e.not_follow_up), e.follow_down/(e.follow_down+e.not_follow_down)])
    ind = np.arange(N)    # the x locations for the groups
    width = 0.35       # the width of the bars: can also be len(x) sequence

    p1 = plt.bar(ind, DOL_Following, width,yerr=DOL_FollowingStd )
    p2 = plt.bar(ind, WDO_Following, width,
                 bottom=DOL_Following, yerr=WDO_FollowingStd)

    plt.ylabel('Percentage')
    plt.title('Movement Trend')
    plt.xticks(ind, ('DOL_Following', 'WDO_Following'))
    # plt.yticks(np.arange(0, 81, 10))
    plt.legend((p1[0], p2[0]), ('Up', 'Down'))

    plt.show()

In [3]:
e = Causality("DOLG20", "WDOG20", "2020-01-06")


In [4]:
import time 
start = time.time()
df= e._files()
end = time.time()
print (end-start)

/home/asimovws/.local/lib/python3.6/site-packages/ipykernel_launcher.py:105: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/asimovws/.local/lib/python3.6/site-packages/ipykernel_launcher.py:106: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/asimovws/.local/lib/python3.6/site-packages/ipykernel_launcher.py:115: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'so

61.04816246032715


In [25]:
df[0]

,index,timestamp,msg_seq_num,ask_0,ask_quantity_0,bid_0,bid_quantity_0,ask_wdo_0,ask_quantity_wdo_0,bid_wdo_0,bid_quantity_wdo_0,spread_dol,spread_wdo,mid_price,mid_price_wdo
18,NaN,NaT,21226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,NaN,NaN
19,1.0,0 days,21227,0.0,0.0,0.0,0.0,0.0,0.0,-0.5,113.0,0.5,1.0,0.0,-0.25


In [6]:
#Lista com msg_seq_number do evento anterior a distorção

Previ = []
for i in df.keys():
    a= df[i].values[0][2]
    Previ.append(a)

In [7]:
#Lista com msg_seq_number de eventos nos quais ocorreram o distorçao
Post = []
for i in df.keys():
    a= df[i].values[1][2]
    Post.append(a)

In [8]:
dh = pd.concat([e.events_wdo,e.events]).sort_values(by='msg_seq_num', kind = 'mergesort').reset_index()


In [9]:
dh['repeat'] = False
dh['repeat'][dh['msg_seq_num'] != dh['msg_seq_num'].shift(-1)] = True

/home/asimovws/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
dh  = dh[dh['repeat']].reset_index()

In [46]:
dh['symbol'][dh['msg_seq_num'].isin(Post)].value_counts()


WDOG20    7949
DOLG20    5014
Name: symbol, dtype: int64

In [30]:
#Check if there are any duplicats in a given column
boolean = dh['msg_seq_num'].duplicated().any() 
boolean


False

In [16]:
#Checa os tipos de eventos logo antes a distorção
event_type = []
for i in Previ:
    event_type.append(dh['event_type'][dh['msg_seq_num'] == i].values[0])

In [17]:
#Checa os tipos de eventos no momento da distorção
event_type_P = []
for i in Post:
    event_type_P.append(dh['event_type'][dh['msg_seq_num'] == i].values[0])

# dh[dh['msg_seq_num'].isin(Post)]['event_type'].value_counts()



In [18]:
dh[dh['msg_seq_num'].isin(Post)]['event_type'].value_counts()

DELETE         6109
DELETE_FROM    4181
CHANGE          188
INSERT           35
Name: event_type, dtype: int64

In [19]:
from collections import Counter
Counter(event_type)

Counter({'DELETE': 4212, 'INSERT': 4570, 'CHANGE': 1538, 'DELETE_FROM': 193})

In [20]:
Counter(event_type_P)

Counter({'INSERT': 35, 'DELETE': 6109, 'DELETE_FROM': 4181, 'CHANGE': 188})

In [11]:
trades = pr.get_parquet(symbol='DOLG20',
                            date='2020-01-06',
                            type_='trades')
md_trades = trades.read().to_pandas()

In [12]:
trades_wdo = pr.get_parquet(symbol='WDOG20',
                            date='2020-01-06',
                            type_='trades')
md_trades_wdo = trades_wdo.read().to_pandas()

In [13]:
all_trades =  pd.concat([md_trades,md_trades_wdo]).sort_values(by='msg_seq_num', kind = 'mergesort').reset_index()

In [14]:
all_trades

,ts,msg_seq_num,symbol,trade_id,price,quantity,trade_ts,buyer,seller,crossed,deleted
0,2020-01-06 12:00:35.881000+00:00,15620,WDOG20,10,4062.0,10.0,12:00:35.878000,308,107,False,None
1,2020-01-06 12:00:35.881000+00:00,15620,WDOG20,20,4062.0,15.0,12:00:35.878000,3,107,False,None
2,2020-01-06 12:00:35.881000+00:00,15620,WDOG20,30,4062.0,1.0,12:00:35.878000,3,3701,False,None
3,2020-01-06 12:00:35.881000+00:00,15620,WDOG20,40,4062.0,10.0,12:00:35.878000,386,3701,False,None
4,2020-01-06 12:00:35.881000+00:00,15620,WDOG20,50,4062.0,1.0,12:00:35.878000,1982,3701,False,None
...,...,...,...,...,...,...,...,...,...,...,...
329239,2020-01-06 21:14:59.050000+00:00,3347315,WDOG20,2969750,4067.5,4.0,21:14:59.049000,3,3,False,None
329240,2020-01-06 21:14:59.050000+00:00,3347315,WDOG20,2969760,4067.5,1.0,21:14:59.049000,3,3,False,None
329241,2020-01-06 21:14:59.591000+00:00,3347323,WDOG20,2969770,4067.5,1.0,21:14:59.591000,3,3,False,None
329242,2020-01-06 21:14:59.591000+00:00,3347323,WDOG20,2969780,4067.5,4.0,21:14:59.591000,3,1618,False,None


In [15]:
list_of_trades = all_trades.groupby('msg_seq_num').sum().index
all_trades.groupby('msg_seq_num').sum()

,trade_id,price,quantity,buyer,seller,crossed
msg_seq_num,,,,,,
15620,1360,64992.0,72.0,5396,26594,0.0
15621,4250,69054.0,26.0,25480,1714,0.0
15622,7140,69054.0,41.0,17181,4102,0.0
15623,10030,69054.0,53.0,5158,29874,0.0
15624,12920,69054.0,64.0,6497,16552,0.0
...,...,...,...,...,...,...
3347287,322630,4067.0,5.0,3,120,0.0
3347288,645290,8136.0,15.0,240,123,0.0
3347315,5939510,8135.0,5.0,6,6,0.0


In [16]:
list_of_trades

Int64Index([  15620,   15621,   15622,   15623,   15624,   15625,   15626,
              15627,   15628,   15629,
            ...
            3347270, 3347273, 3347274, 3347281, 3347282, 3347287, 3347288,
            3347315, 3347323, 3347334],
           dtype='int64', name='msg_seq_num', length=155635)

In [17]:
Msg_T=list(set(list_of_trades).intersection(set(Post)))

In [169]:
dl= dh

In [83]:
B = dh[dh['msg_seq_num'].isin(Msg_T)]['symbol'].value_counts()

In [39]:
print('Wdo : {} %, Dol: {} %'.format(4697/6142, 2642/4371))

Wdo : 0.7647346141322044 %, Dol: 0.6044383436284603 %


In [59]:
print('Wdo : {} %, Dol: {} %'.format(4804/6252, 3644/5972))

Wdo : 0.7683941138835573 %, Dol: 0.6101808439383791 %


In [75]:
print('Wdo : {} %, Dol: {} %'.format(5597/7949, 2946/5014))

Wdo : 0.704113724996855 %, Dol: 0.587554846429996 %


In [97]:
pd.DataFrame(['2020-01-02', A[0], B[0], A[1], B[1], B[0]/A[0], B[1]/A[1]]).T.rename({0: 'Date', 1:'WDO_Init', 2:'WDO_Trades', 3: 'DOL_Init', 4:'DOL_Trades', 5: 'WDO_%_Trades', 6:'DOL_%_Trades' }, axis =1)

,WDO_Init,WDO_Trades,DOL_Init,DOL_Trades,WDO_%_Trades,DOL_%_Trades,6
0,2020-01-02,7949,5597,5014,2946,0.704114,0.587555


In [2]:
def get_trades(dol, wdo, date_):
    trades = pr.get_parquet(symbol=dol,
                                date=date_,
                                type_='trades')
    md_trades = trades.read().to_pandas()
    trades_wdo = pr.get_parquet(symbol=wdo,
                                date=date_,
                                type_='trades')
    md_trades_wdo = trades_wdo.read().to_pandas()
    return md_trades, md_trades_wdo 



In [3]:
DATE = ['2020-01-02',
    '2020-01-06',
    '2020-01-03',
    '2020-01-07',
    '2020-01-16',
    '2020-01-09',
    '2020-01-10',
    '2020-01-08']

In [4]:
j = 0
dados= {}
for k in DATE:
    e = Causality("DOLG20", "WDOG20", k)

    md_trades, md_trades_wdo = get_trades("DOLG20", "WDOG20", k)

    df = e._files()

    Post = []
    for i in df.keys():
        a= df[i].values[1][2]
        Post.append(a)

    dh = pd.concat([e.events_wdo,e.events]).sort_values(by='msg_seq_num', kind = 'mergesort').reset_index()   
    dh['repeat'] = False
    dh['repeat'][dh['msg_seq_num'] != dh['msg_seq_num'].shift(-1)] = True
    dh  = dh[dh['repeat']].reset_index()
    TOTAL = dh['symbol'][dh['msg_seq_num'].isin(Post)].value_counts()

    all_trades =  pd.concat([md_trades,md_trades_wdo]).sort_values(by='msg_seq_num', kind = 'mergesort').reset_index()
    list_of_trades = all_trades.groupby('msg_seq_num').sum().index
    Msg_T=list(set(list_of_trades).intersection(set(Post)))
    TRADES = dh[dh['msg_seq_num'].isin(Msg_T)]['symbol'].value_counts()

    dados[j] = pd.DataFrame([k, TOTAL[0], TRADES[0], TOTAL[1], TRADES[1], TRADES[0]/TOTAL[0], TRADES[1]/TOTAL[1]]).T.rename({0: 'Date', 1:'WDO_Init', 2:'WDO_Trades', 3: 'DOL_Init', 4:'DOL_Trades', 5: 'WDO_%_Trades', 6:'DOL_%_Trades' }, axis =1)
    j+=1

/home/asimovws/.local/lib/python3.6/site-packages/ipykernel_launcher.py:105: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/asimovws/.local/lib/python3.6/site-packages/ipykernel_launcher.py:106: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/asimovws/.local/lib/python3.6/site-packages/ipykernel_launcher.py:115: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'so

In [10]:
all_ = pd.concat(dados)

In [37]:
all_

,,Date,WDO_Init,WDO_Trades,DOL_Init,DOL_Trades,WDO_%_Trades,DOL_%_Trades
0,0,2020-01-02,6142,4697,4371,2642,0.764735,0.604438
1,0,2020-01-06,7949,5597,5014,2946,0.704114,0.587555
2,0,2020-01-03,6252,4804,5972,3644,0.768394,0.610181
3,0,2020-01-07,7964,5422,6500,3992,0.680814,0.614154
4,0,2020-01-16,7973,5605,5541,3200,0.702998,0.577513
5,0,2020-01-09,8923,6230,6686,3954,0.698196,0.591385
6,0,2020-01-10,8858,6357,5769,3296,0.717656,0.57133
7,0,2020-01-08,11377,8374,7693,4202,0.736046,0.546211


In [143]:
all_[['WDO_Init', 'WDO_Trades', 'DOL_Init', 'DOL_Trades',
       'WDO_%_Trades', 'DOL_%_Trades']].mean()

WDO_Init        8179.750000
WDO_Trades      5885.750000
DOL_Init        5943.250000
DOL_Trades      3484.500000
WDO_%_Trades       0.721619
DOL_%_Trades       0.587846
dtype: float64

In [23]:
move = pd.concat(df)

In [43]:
dol_down = move[move['movement_dol'] == -1]['msg_seq_num']

In [44]:
dol_up = move[move['movement_dol'] == 1]['msg_seq_num']

In [45]:
wdo_down = move[move['movement_wdo'] == -1]['msg_seq_num']

In [54]:
wdo_up = move[move['movement_wdo'] == 1]['msg_seq_num']


In [20]:
all_trades

,ts,msg_seq_num,symbol,trade_id,price,quantity,trade_ts,buyer,seller,crossed,deleted
0,2020-01-06 12:00:35.881000+00:00,15620,WDOG20,10,4062.0,10.0,12:00:35.878000,308,107,False,None
1,2020-01-06 12:00:35.881000+00:00,15620,WDOG20,20,4062.0,15.0,12:00:35.878000,3,107,False,None
2,2020-01-06 12:00:35.881000+00:00,15620,WDOG20,30,4062.0,1.0,12:00:35.878000,3,3701,False,None
3,2020-01-06 12:00:35.881000+00:00,15620,WDOG20,40,4062.0,10.0,12:00:35.878000,386,3701,False,None
4,2020-01-06 12:00:35.881000+00:00,15620,WDOG20,50,4062.0,1.0,12:00:35.878000,1982,3701,False,None
...,...,...,...,...,...,...,...,...,...,...,...
329239,2020-01-06 21:14:59.050000+00:00,3347315,WDOG20,2969750,4067.5,4.0,21:14:59.049000,3,3,False,None
329240,2020-01-06 21:14:59.050000+00:00,3347315,WDOG20,2969760,4067.5,1.0,21:14:59.049000,3,3,False,None
329241,2020-01-06 21:14:59.591000+00:00,3347323,WDOG20,2969770,4067.5,1.0,21:14:59.591000,3,3,False,None
329242,2020-01-06 21:14:59.591000+00:00,3347323,WDOG20,2969780,4067.5,4.0,21:14:59.591000,3,1618,False,None


In [67]:
all_grouped = all_trades[all_trades['msg_seq_num'].isin(Msg_T)].groupby(['msg_seq_num','symbol']).agg({'price' : 'mean','quantity': 'sum'}).reset_index()




In [68]:
all_grouped[all_grouped['symbol'] == 'WDOG20'][['price', 'quantity']].median()


price       4064.0
quantity      13.0
dtype: float64

In [69]:
all_grouped[all_grouped['symbol'] == 'DOLG20'][['price', 'quantity']].median()


price       4063.5
quantity      10.0
dtype: float64

In [71]:
all_trades_grouped = all_trades.groupby(['msg_seq_num','symbol']).agg({'price' : 'mean','quantity': 'sum'}).reset_index()


In [72]:
all_trades_grouped[all_trades_grouped['symbol'] == 'WDOG20'][['price', 'quantity']].median()

price       4064.0
quantity       2.0
dtype: float64

In [73]:
all_trades_grouped[all_trades_grouped['symbol'] == 'DOLG20'][['price', 'quantity']].median()

price       4063.205882
quantity       5.000000
dtype: float64

In [77]:
all_grouped_without = all_trades[~all_trades['msg_seq_num'].isin(Msg_T)].groupby(['msg_seq_num','symbol']).agg({'price' : 'mean','quantity': 'sum'}).reset_index()


In [78]:
all_grouped_without[all_grouped_without['symbol'] == 'WDOG20'][['price', 'quantity']].median()

price       4064.0
quantity       2.0
dtype: float64

In [79]:
all_grouped_without[all_grouped_without['symbol'] == 'DOLG20'][['price', 'quantity']].median()

price       4063.0
quantity       5.0
dtype: float64

In [32]:
# index_ = dh[dh['msg_seq_num'] == i].index[0]

In [34]:
dict_ = {}
j = 0 
for i in wdo_up:
    index_ = dh[dh['msg_seq_num'] == i].index[0]
    x = dh['event_type'].values[index_-200:index_-1]
    dict_[j] =pd.DataFrame(np.array(np.unique(x, return_counts=True)).T)
    j+=1

In [117]:
pd.concat(dict_).reset_index().groupby(0).sum()

,level_0,level_1
0,,
CHANGE,3241661,0
DELETE,3247426,2545
DELETE_FROM,2683071,4140
INSERT,3247426,7166


In [123]:
pd.concat(dict_).groupby(0).sum()

,1
0,
CHANGE,135183
DELETE,307098
DELETE_FROM,12569
INSERT,336175


In [121]:
pd.concat(dict_).groupby(0).sum()

,1
0,
CHANGE,95554
DELETE,181376
DELETE_FROM,6299
INSERT,207306


# Random Forests

In [ ]:
# dh['predic'] = [1 if x in (Msg_T) else 0 for x in dh['msg_seq_num']]
# dl['msg_seq_num'].apply(lambda x: 1 if x in (Msg_T) else 0)

In [18]:
dh['predic'] = 0

In [19]:
dh.loc[dh['msg_seq_num'].isin(Previ), 'predic'] = 1

In [20]:
dh.drop(['index'], axis =1, inplace = True)
dh.reset_index(inplace=True)

In [21]:
dh.dropna(inplace = True)

In [22]:
everything = pd.get_dummies(dh[['symbol', 'side', 'event_type', 'quantity', 'price', 'predic']])

In [62]:
everything

,quantity,price,predic,symbol_DOLG20,symbol_WDOG20,side_A,side_B,event_type_CHANGE,event_type_INSERT
0,27.0,4020.0,0,0,1,1,0,0,1
1,5.0,4063.5,0,1,0,1,0,0,1
2,20.0,4027.0,0,1,0,0,1,0,1
3,1.0,4052.0,0,0,1,0,1,0,1
4,1.0,4050.5,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...
2221381,5.0,4067.0,0,0,1,0,1,0,1
2221382,5.0,4067.0,0,0,1,0,1,0,1
2221383,4.0,4067.5,0,0,1,1,0,0,1
2221384,4.0,4063.0,0,0,1,0,1,1,0


In [49]:
everything = everything.reset_index().drop({'level_0', 'index'}, axis =1)

In [50]:
X = everything[['symbol_DOLG20', 'symbol_WDOG20','side_A', 'side_B', 'event_type_CHANGE', 'event_type_INSERT',  'quantity', 'price']]
y = everything[['predic']]

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.50, random_state=42, stratify=y)


In [57]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [86]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier


clf = RandomForestClassifier(n_estimators = 200, random_state=0, class_weight='balanced')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


/home/asimovws/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [89]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_pred,y_test))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[915270   1961]
 [191494   1968]]
              precision    recall  f1-score   support

           0       1.00      0.83      0.90   1106764
           1       0.01      0.50      0.02      3929

    accuracy                           0.83   1110693
   macro avg       0.50      0.66      0.46   1110693
weighted avg       0.99      0.83      0.90   1110693

0.8258249579316697


In [88]:
from sklearn import metrics


print("Accuracy:",metrics.accuracy_score(y_test, y_pred))



Accuracy: 0.8258249579316697


In [84]:
feature_imp = pd.Series(clf.feature_importances_,index=X.columns).sort_values(ascending=False)
feature_imp



price                0.436431
quantity             0.410972
symbol_WDOG20        0.039976
event_type_INSERT    0.028792
event_type_CHANGE    0.027397
symbol_DOLG20        0.026928
side_A               0.015072
side_B               0.014431
dtype: float64

# Gradient Boosting


In [80]:
gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=0.05, max_features=2, max_depth=2, random_state=0, class_weight='balanced')
gb_clf.fit(X_train, y_train)

feature_imp = pd.Series(gb_clf.feature_importances_,index=X.columns).sort_values(ascending=False)
feature_imp


TypeError: __init__() got an unexpected keyword argument 'class_weight'

In [68]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier()
xgb_clf.fit(X_train, y_train)

score = xgb_clf.score(X_val, y_val)
print(score)


ModuleNotFoundError: No module named 'xgboost'